In [1]:
import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import pandas as pd
import os
from tensorflow.keras.models import Model

from models.resnet import build_resnet50
from models.densenet import build_densenet121
from models.darknet19 import darknet19_detection

/vol/tensusers2/sdooper/Anaconda3/envs/dam/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/vol/tensusers2/sdooper/Anaconda3/envs/dam/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/vol/tensusers2/sdooper/Anaconda3/envs/dam/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/vol/tensusers2/sdoope

In [2]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

In [3]:
#necessary classes for model loading

class BaseFeatureExtractor(object):
    """docstring for ClassName"""

    # to be defined in each subclass
    def __init__(self, input_size):
        raise NotImplementedError("error message")

    # to be defined in each subclass
    def normalize(self, image):
        raise NotImplementedError("error message")

    def get_output_shape(self):
        return self.feature_extractor.get_output_shape_at(-1)[1:3]

    def extract(self, input_image):
        return self.feature_extractor(input_image)
    
    
class ResNet50Feature(object):
    
    def __init__(self, input_size):
        params = {'num_classes': 2, 'channels': ['B4','B3','B2','MNDWI','AVE'], 'weights':'imagenet', 'top':False}
        self.resnet50 = build_resnet50(**params)
        

class DenseNet121Feature(object):
    
    def __init__(self, input_size):
        params = {'num_classes': 2, 'channels': ['B4','B3','B2','MNDWI','AVE'], 'weights':'imagenet', 'top':False}
        self.densenet121 = build_densenet121(**params)
        
class Darknet19Feature(object):
    
    def __init__(self, input_size):
        params = {'num_classes': 2, 'channels': ['B4','B3','B2','MNDWI','AVE'], 'weights':'imagenet', 'top':False}
        self.darknet19 = darknet19_detection(**params)
    

In [8]:
model = Darknet19Feature(input_size=[257,257])
model1 = model.darknet19

instantiated model with weights: imagenet
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputlayer_0 (InputLayer)    [(None, None, None, 5)]   0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, None, None, 3)     18        
_________________________________________________________________
Full_YOLO_backend (Model)    (None, None, None, 1024)  50547936  
Total params: 50,547,954
Trainable params: 50,527,282
Non-trainable params: 20,672
_________________________________________________________________


In [11]:
model1.summary()
print(model1.layers[1].get_weights())
model1.load_weights('backend_weights/darknet19_backend.h5')
print(model1.layers[1].get_weights())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputlayer_0 (InputLayer)    [(None, None, None, 5)]   0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, None, None, 3)     18        
_________________________________________________________________
Full_YOLO_backend (Model)    (None, None, None, 1024)  50547936  
Total params: 50,547,954
Trainable params: 50,527,282
Non-trainable params: 20,672
_________________________________________________________________
[array([[[[-0.7001643 ,  0.10685214, -0.06933999],
         [-0.7978253 , -0.04125755,  0.4032792 ],
         [-0.57961106, -0.3013558 ,  0.2384147 ],
         [-0.01017861, -0.70373017,  0.70956975],
         [ 0.31965765,  0.02333218, -0.5410377 ]]]], dtype=float32), array([ 0.06248161,  0.06684788, -0.00197338], dtype=float32)]
[array([[[[-0.7001643 ,  0.10685214, -0

In [7]:
model1.save_weights('darknet19_backend.h5')